<h2>業種別で重回帰分析をおこないリターンの予測をしよう！</h3>

In [1]:
# 今回使うライブラリ
import pandas as pd
import numpy  as np
from numpy import nan # 欠損値
from sklearn import linear_model #機械学習ライブラリより線形モデルを取得
import statsmodels.api as smf # 統計量計算
import matplotlib.pyplot as plt # 描画ライブラリ
import seaborn as sns # 描画ライブラリ

<h3 id = "データを読み込む">データを読み込む</h3>

In [ ]:
processed_data = pd.read_csv("ProcessedData.csv")
processed_data

<h3 id = "業種別でデータを取得する">業種別でデータを取得する</h3>

In [ ]:
# 【120以上の業種】
# '建設業': 158
#  '食料品': 122
#  '化学': 210
#  '機械': 228
#  '電気機器': 243
#  '卸売業': 317
#  '小売業': 348
#  '不動産業': 132
#  'サービス業': 470
#  '情報・通信業': 462

In [ ]:
name =input()


with open(f"ticker_list/{name}.txt" ,mode = "r") as f:
            file = f.read()
ticker_list_three = file.split("\n")
    
industry_list = []
for code in ticker_list_three :
    ticker_num = code.split(",")
    for count in ticker_num :
        industry_list.append(count)

        
scr_industry = pd.DataFrame()
for i in  industry_list:
    #銘柄コードが" "の文字列を特定し、エラーを出さないために弾く
    if i.isdecimal() :
        i_int =int(i) 
        #memoryをもちいて銘柄コードを抽出する(参照渡しの回避)
        code_data = processed_data[processed_data["code"] == i_int ]
        scr_industry = pd.concat([scr_industry,code_data])


scr_industry

<h3 id = "統計量を確認する">統計量を確認する</h3>

In [ ]:
clf = linear_model.LinearRegression()

# データフレームで分析に使わないcolumnsを除去               
use_data = scr_industry.drop(["code","company"], axis = 1)

#  データフレームの各列を正規化                                                               全ての列がnanのものを除去
use_data_new = use_data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x))).dropna(how='all', axis=1)


#説明変数を作成           #目的変数を除去
use_data_new_except_criterion = use_data_new.drop(use_data_new.iloc[:,[-1]], axis = 1)
X = use_data_new_except_criterion.values 
                             
#目的変数を作成 
Y = use_data_new.iloc[:,-1].values
                                                  

#統計量を作成
model = smf.OLS(Y,X) 
result = model.fit()
result.summary()  

<h2>データ処理をおこない再度統計量を確認しよう！</h2>
<h3>P値の結果によるデータ処理をおこない再度統計量を確認する</h3>

In [ ]:
# 予測モデルを作成
clf.fit(X, Y)

# 偏回帰係数
print(pd.DataFrame({"Name":use_data_new_except_criterion.columns,
                    "Coefficients":np.abs(clf.coef_)}))

# 切片 (誤差)
print(clf.intercept_)   

In [ ]:
use_data_re = use_data_new[[
                           "net_d/e_ratio",
                           "doe"  
                           ]]
X = use_data_re.values
#Yは目的変数のため変更しない

#再度統計量を出す
model = smf.OLS(Y,X) 
result = model.fit()
result.summary()

<h3>多重共線性によるデータ処理をおこない再度統計量を確認する</h3>

In [ ]:
clf.fit(X, Y)
colormap = plt.cm.RdBu
# フォントの設定
plt.rcParams['font.family'] = 'Yu Mincho'
# 描画サイズの設定
plt.figure(figsize=(14,12))
# タイトル
plt.title('証券データの相関行列', y=1.05, size=15)
# ヒートマップの描画
sns.heatmap(use_data_re.astype(float).corr(),linewidths=0.1,vmax=1.0, 
         square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
use_data_re = use_data_new[[
                           "net_d/e_ratio",
                           "doe"  
                           ]]
X = use_data_re.values
#Yは目的変数のため変更しない

#再度統計量を出す
model = smf.OLS(Y,X) 
result = model.fit()
result.summary()

<h2>予測値をもとに企業をスクリーニングしよう！</h2>
<h3>全ての企業の予測値を求める</h3>

In [ ]:
#説明変数の実測値
measured_valueE = pd.DataFrame()
for i in use_data_re.iloc[0,:].index :
    data_merge = processed_data[[i]]
    measured_valueE = pd.concat([measured_valueE,data_merge],axis=1)

#係数の計算
clf.fit(X, Y)

#説明変数の予測値の係数
predicted_valueE = pd.DataFrame({"Name": use_data_re.columns,"Coefficients": clf.coef_ })

#説明変数の予測値の切片
predicted_valueI = clf.intercept_ 

############################################################

#実測値と予測値の係数の掛け算をおこない予測値を出す
E = list(map(lambda x: sum(x *  predicted_valueE.iloc[:,1].values) + predicted_valueI ,measured_valueE.values))
E

<h3>予測値をもとに企業をスクリーニングして出力する</h3>

In [ ]:
#回帰式の予測値より現実のデータが低くなっている上位10％のインデックスと、その乖離度を出す。
from copy import deepcopy as cp

Y= processed_data.iloc[:,-1].values


scr_list = []

for i in range(len(E)):
    #予測値が実測値より大きいか判定 
    if E[i] >= Y[i]:
        scr_list.append([i,E[i] - Y[i]])

can_add = True
while can_add :
    f = 0
    for i in range(len(scr_list) - 1):
        if scr_list [i][1] < scr_list[i + 1][1]:
            save = cp(scr_list[i])
            scr_list[i] = cp(scr_list[i + 1])
            scr_list[i + 1] = cp(save)
            f = 1
    if f == 0 :
        can_add = False
        
        

#乖離度上位10％を抽出 
sampling = scr_list[:round(len(scr_list)*0.1)]
if len(sampling) == 0:
     print("期待利益を見込める企業が見つかりませんでした")
    

else:
    out_put = pd.DataFrame()

    times = 0
    for i in [sampling[rank][0] for rank in range(len(sampling))]:
        times +=1
        relay = processed_data.iloc[[i]]
        relay["expect_rate"] = E[i]
        out_put = pd.concat([out_put,relay])
        if times == 5:
            break
            
    #上位10％からtop有望銘柄を最大5つ出力  
    top_company = out_put.iloc[:,[0,1,-2,-1]]

top_company